In [1]:
import pandas as pd
from pathlib import Path

In [2]:
!pip install fastai2>=0.0.11 graphviz ipywidgets matplotlib nbdev>=0.2.12 pandas scikit_learn azure-cognitiveservices-search-imagesearch sentencepiece


In [3]:
from fastai2.vision.all import *
from sklearn.metrics import roc_auc_score


RuntimeError: tensor.H is only supported on matrices (2-D tensors). Got 1-D tensor.

In [4]:
data_path = Path("../input/plant-pathology-2020-fgvc7/")


In [5]:
df = pd.read_csv(data_path/"train.csv")


In [6]:
df.head()


,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,1,0
1,Train_1,1,0,0,0
2,Train_2,0,0,1,0
3,Train_3,1,0,0,0
4,Train_4,0,0,1,0


In [7]:
df.iloc[:, 1:].sum(axis=1).value_counts()


1    1638
Name: count, dtype: int64

In [8]:
imglabels = list(df.columns[1:])


In [9]:
df["labels"] = df.apply(lambda x: imglabels[x.values[1:].argmax()], axis=1)


In [10]:
df.head()


,image_id,healthy,multiple_diseases,rust,scab,labels
0,Train_0,0,0,1,0,rust
1,Train_1,1,0,0,0,healthy
2,Train_2,0,0,1,0,rust
3,Train_3,1,0,0,0,healthy
4,Train_4,0,0,1,0,rust


In [11]:
dls = ImageDataLoaders.from_df(df,
                               path=data_path, 
                               suff=".jpg", 
                               folder="images",
                               label_col="labels",
                               item_tfms=Resize(224))


NameError: name 'ImageDataLoaders' is not defined

In [12]:
dls.show_batch()


NameError: name 'dls' is not defined

In [13]:
def mean_roc_auc(preds, targets, num_cols=4):
    """The competition metric
    
    Quoting: 'Submissions are evaluated on mean column-wise ROC AUC. 
    In other words, the score is the average of the individual AUCs 
    of each predicted column. '
    
    Unfortunately, we cannot use in validation, as it can happen that
    all files in a batch has the same label, and ROC is undefined
    """
    aucs = []
    preds = preds.detach().cpu().numpy()
    targets = targets.detach().cpu().numpy()
    
    for i in range(num_cols):
        # grab a column from the networks output
        cpreds = preds[:, i]
        # see which objects have the i-th label
        ctargets = [x == i for x in targets]
        aucs.append(roc_auc_score(ctargets, cpreds))
    return sum(aucs) / num_cols


In [14]:
learn = cnn_learner(dls, resnet34, metrics=[error_rate])


NameError: name 'cnn_learner' is not defined

In [15]:
learn.fine_tune(5)


NameError: name 'learn' is not defined

In [16]:
test_image_ids = [img.split(".")[0] for img in os.listdir(data_path/"images") if img.startswith("Test")]
test_images = [data_path/"images"/f"{img}.jpg" for img in test_image_ids]
preds = learn.get_preds(dl=dls.test_dl(test_images, shuffle=False, drop_last=False))


NameError: name 'os' is not defined

In [17]:
# ensure that the order of columns in preds matches the imglabels
preds = preds[0].cpu().numpy()
vocab = list(dls[0].dataset.vocab)
column_permutation = [vocab.index(l) for l in imglabels]
preds = preds[:, column_permutation]

submission = pd.DataFrame()
submission["image_id"] = test_image_ids
for i in range(len(imglabels)):
    submission[imglabels[i]] = preds[:, i]
submission.to_csv("submission.csv", index=False)


NameError: name 'preds' is not defined

In [18]:
submission.head(10)


NameError: name 'submission' is not defined